# Analysis of the homogeneity of neighborhood in New York

In [1]:
import numpy as np
import pandas as pd
!pip install folium
import folium 
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests

from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

     |████████████████████████████████| 94 kB 5.1 MB/s  eta 0:00:01


In [2]:
# Four square credential and version
CLIENT_ID = "O4QRQSIIHF54HU10IK1R44W4YSY2TLZDRBHOT2FYBIT4XRFK"
CLIENT_SECRET= "Z0X4BRVIPSLPYHOGP13SPODQERWWBPQWVT2WNL453COBLJLX"
VERSION = '20180605'
LIMIT= 100

In [3]:
# Download the data
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


In [4]:
#load the data 
with open ('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [5]:
# Create the data frame
neighborhoods_data = newyork_data["features"]
column_names =["Borough", "Neighborhood", "Latitude", 'Longitude']
neighborhoods= pd.DataFrame(columns= column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data["properties"]["borough"]
    neighborhood_name = data["properties"]["name"]
    
    neighborhood_latlon = data["geometry"]["coordinates"]
    neighborhood_lat    = neighborhood_latlon[1]
    neighborhood_lon    = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({"Borough": borough,
                                         "Neighborhood": neighborhood_name,
                                         "Latitude": neighborhood_lat,
                                         'Longitude': neighborhood_lon}, ignore_index=True)

In [6]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


## working the Neighborhood

In [7]:
# List of neighborhoods
Queens_data    = neighborhoods[neighborhoods["Borough"]=='Queens'].reset_index(drop=True)
statenIsland   = neighborhoods[neighborhoods["Borough"]=='Staten Island'].reset_index(drop=True)
manhattan_data = neighborhoods[neighborhoods["Borough"]=='Manhattan'].reset_index(drop=True)
print("Neighborhood ok!")

Neighborhood ok!


## I. Queens

In [8]:
address = 'Queens, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location   = geolocator.geocode(address)
latitude   = location.latitude
longitude  = location.longitude

In [9]:
#Map the neighborhood
map_Queens = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, label in zip(Queens_data["Latitude"], Queens_data["Longitude"], Queens_data["Neighborhood"]):
    label = folium.Popup(label, parse_html= True)
    folium.CircleMarker(
    [lat, lng],
    radius = 5,
    popup = label,
    color="blue",
    fill = False,
    parse_html=False).add_to(map_Queens)
    
map_Queens

In [10]:
# get the neighborhood's name with their latitude and longitude 
neighborhood_name = Queens_data.loc[0,"Neighborhood"]
neighborhood_latitude = Queens_data.loc[0, "Latitude"]
neighborhood_longitude= Queens_data.loc[0, "Longitude"]

let get the top 100 venues that are in Marble Hill within a radius of 500 m

In [11]:
# get the URL
LIMIT = 100
radius= 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
#get the result
results = requests.get(url).json()

In [12]:
# extract tge category of the venue
def get_category_type (row):
    try:
        categories_list=row["categories"]
    except:
        categories_list=row['venue.categories']
    if len (categories_list)==0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
# put the json data into a pandas dataframe
venues = results["response"]["groups"][0]['items']
nearby_venues = json_normalize(venues)

filtered_columns = ["venue.name", "venue.categories","venue.location.lat","venue.location.lng"]
nearby_venues    = nearby_venues.loc[:, filtered_columns]

nearby_venues["venue.categories"] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns =[col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Favela Grill,Brazilian Restaurant,40.767348,-73.917897
1,Orange Blossom,Gourmet Shop,40.769856,-73.917012
2,Titan Foods Inc.,Gourmet Shop,40.769198,-73.919253
3,CrossFit Queens,Gym,40.769404,-73.918977
4,Simply Fit Astoria,Gym,40.769114,-73.912403


## I.1. Explore the neighborhood

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the url
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # make a Get request
        results= requests.get(url).json()["response"]["groups"][0]["items"]
        
        # relevant information
        venues_list.append([(name,
                             lat, 
                             lng, 
                             v['venue']["name"], 
                             v["venue"]["location"]["lat"], 
                             v["venue"]["location"]["lng"],
                             v["venue"]["categories"][0]["name"]) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns =["Neighborhood",
                            "Neighborhood Latitude",
                           "Neighborhood Longitude",
                           'Venue',
                           "Venue Latitude",
                           "Venue Longitude",
                           "Venue Category"]
    return(nearby_venues)

In [15]:
# for each neighborhood
Queens_venues = getNearbyVenues(Queens_data["Neighborhood"], Queens_data["Latitude"], Queens_data["Longitude"])

Astoria
Woodside
Jackson Heights
Elmhurst
Howard Beach
Corona
Forest Hills
Kew Gardens
Richmond Hill
Flushing
Long Island City
Sunnyside
East Elmhurst
Maspeth
Ridgewood
Glendale
Rego Park
Woodhaven
Ozone Park
South Ozone Park
College Point
Whitestone
Bayside
Auburndale
Little Neck
Douglaston
Glen Oaks
Bellerose
Kew Gardens Hills
Fresh Meadows
Briarwood
Jamaica Center
Oakland Gardens
Queens Village
Hollis
South Jamaica
St. Albans
Rochdale
Springfield Gardens
Cambria Heights
Rosedale
Far Rockaway
Broad Channel
Breezy Point
Steinway
Beechhurst
Bay Terrace
Edgemere
Arverne
Rockaway Beach
Neponsit
Murray Hill
Floral Park
Holliswood
Jamaica Estates
Queensboro Hill
Hillcrest
Ravenswood
Lindenwood
Laurelton
Lefrak City
Belle Harbor
Rockaway Park
Somerville
Brookville
Bellaire
North Corona
Forest Hills Gardens
Jamaica Hills
Utopia
Pomonok
Astoria Heights
Hunters Point
Sunnyside Gardens
Blissville
Roxbury
Middle Village
Malba
Hammels
Bayswater
Queensbridge


In [16]:
print(Queens_venues.shape)
Queens_venues.head()

(2103, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
2,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop
3,Astoria,40.768509,-73.915654,CrossFit Queens,40.769404,-73.918977,Gym
4,Astoria,40.768509,-73.915654,Simply Fit Astoria,40.769114,-73.912403,Gym


## I.2. Analyse each neighborhood

In [17]:
#one hot encoding
Queens_onehot = pd.get_dummies(Queens_venues[["Venue Category"]], prefix="", prefix_sep="")

#add neighborhood column back to to dataframe
Queens_onehot["Neighborhood"]= Queens_venues["Neighborhood"]
fixed_columns = [Queens_onehot.columns[-1]]+ list(Queens_onehot.columns[:-1])
Queens_onehot = Queens_onehot[fixed_columns]

print(Queens_onehot.shape)
Queens_onehot.head()

(2103, 269)


,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# grouping rows by neighborhood
Queens_grouped = Queens_onehot.groupby("Neighborhood").mean().reset_index()
print(Queens_grouped.shape)
Queens_grouped.head()

(81, 269)


,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Women's Store
0,Arverne,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.050000,0.000000
1,Astoria,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.010101,0.000000,0.0,0.0,0.0,0.000000,0.0,0.010101,0.000000
2,Astoria Heights,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
3,Auburndale,0.0,0.000000,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
4,Bay Terrace,0.0,0.027027,0.0,0.054054,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.027027,0.0,0.0,0.0,0.027027,0.0,0.000000,0.054054


In [19]:
# Neighborhood with the top 5 most common venues
num_top_venues = 5

for i in Queens_grouped["Neighborhood"]:
    print("---" + i + "---")
    temp = Queens_grouped[Queens_grouped["Neighborhood"]==i].T.reset_index()
    temp.columns = ["venue", "freq"]
    temp = temp.iloc[1:]
    temp["freq"]= temp["freq"].astype(float)
    temp= temp.round({"freq":2})
    print (temp.sort_values("freq", ascending=False).reset_index(drop=True).head(num_top_venues))
    print("\n")

---Arverne---
             venue  freq
0        Surf Spot  0.20
1   Sandwich Place  0.10
2    Metro Station  0.10
3      Coffee Shop  0.05
4  Thai Restaurant  0.05


---Astoria---
                       venue  freq
0                        Bar  0.07
1  Middle Eastern Restaurant  0.06
2           Greek Restaurant  0.04
3   Mediterranean Restaurant  0.04
4         Seafood Restaurant  0.04


---Astoria Heights---
                venue  freq
0  Chinese Restaurant  0.13
1  Italian Restaurant  0.07
2              Hostel  0.07
3         Bus Station  0.07
4              Museum  0.07


---Auburndale---
                  venue  freq
0    Italian Restaurant  0.11
1    Athletics & Sports  0.05
2        Gymnastics Gym  0.05
3  Fast Food Restaurant  0.05
4           Supermarket  0.05


---Bay Terrace---
            venue  freq
0  Clothing Store  0.11
1   Women's Store  0.05
2            Bank  0.05
3  Cosmetics Shop  0.05
4      Shoe Store  0.05


---Bayside---
               venue  freq
0           

In [20]:
# put into pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0: num_top_venues]

In [21]:
num_top_venues = 10
indicators = ["st", "nd", "rd"]

columns = ["Neighborhood"]
for i in np.arange(num_top_venues):
    try:
        columns.append("{}{} Most Common Venue".format(i+1, incdicators[i]))
    except:
        columns.append("{}th Most Common venue".format(i+1))

# create dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted["Neighborhood"] = Queens_grouped["Neighborhood"]

for i in np.arange(Queens_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[i, 1:] = return_most_common_venues(Queens_grouped.iloc[i, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1th Most Common venue,2th Most Common venue,3th Most Common venue,4th Most Common venue,5th Most Common venue,6th Most Common venue,7th Most Common venue,8th Most Common venue,9th Most Common venue,10th Most Common venue
0,Arverne,Surf Spot,Metro Station,Sandwich Place,Donut Shop,Bus Stop,Café,Board Shop,Beach,Thai Restaurant,Coffee Shop
1,Astoria,Bar,Middle Eastern Restaurant,Seafood Restaurant,Hookah Bar,Café,Bakery,Greek Restaurant,Mediterranean Restaurant,Indian Restaurant,Food Truck
2,Astoria Heights,Chinese Restaurant,Pizza Place,Italian Restaurant,Bus Station,Bowling Alley,Museum,Supermarket,Burger Joint,Bakery,Food
3,Auburndale,Italian Restaurant,Mattress Store,Pet Store,Deli / Bodega,Comic Shop,Discount Store,Noodle House,Fast Food Restaurant,Mobile Phone Shop,Miscellaneous Shop
4,Bay Terrace,Clothing Store,Women's Store,Donut Shop,Shoe Store,Mobile Phone Shop,Bank,Kids Store,Cosmetics Shop,American Restaurant,Video Game Store


## I.3. Cluster neighborhoods

In [22]:
kclusters = 4

Queens_grouped_clustering = Queens_grouped.drop("Neighborhood", 1)

# rum Kmean clustering
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(Queens_grouped_clustering)

#add labels
neighborhoods_venues_sorted.insert(0, "Cluster Labels", kmeans.labels_)
Queens_merged = Queens_data

Queens_merged = Queens_merged.join(neighborhoods_venues_sorted.set_index("Neighborhood"), on="Neighborhood")
Queens_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1th Most Common venue,2th Most Common venue,3th Most Common venue,4th Most Common venue,5th Most Common venue,6th Most Common venue,7th Most Common venue,8th Most Common venue,9th Most Common venue,10th Most Common venue
0,Queens,Astoria,40.768509,-73.915654,1,Bar,Middle Eastern Restaurant,Seafood Restaurant,Hookah Bar,Café,Bakery,Greek Restaurant,Mediterranean Restaurant,Indian Restaurant,Food Truck
1,Queens,Woodside,40.746349,-73.901842,1,Grocery Store,Bakery,Thai Restaurant,Filipino Restaurant,Latin American Restaurant,Bar,American Restaurant,Pub,Donut Shop,Sandwich Place
2,Queens,Jackson Heights,40.751981,-73.882821,1,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Grocery Store,Bakery,Thai Restaurant,Mobile Phone Shop,Spanish Restaurant,Supermarket,Supplement Shop
3,Queens,Elmhurst,40.744049,-73.881656,1,Thai Restaurant,Mexican Restaurant,Chinese Restaurant,Vietnamese Restaurant,Snack Place,Seafood Restaurant,Sushi Restaurant,Bar,Donut Shop,Malay Restaurant
4,Queens,Howard Beach,40.654225,-73.838138,1,Pharmacy,Italian Restaurant,Fast Food Restaurant,Sandwich Place,Ice Cream Shop,Spa,Sushi Restaurant,Clothing Store,Supermarket,Chinese Restaurant


In [23]:
# create a map
map_clusters = folium.Map(Location=[latitude, longitude], zoom_start= 10)

x= np.arange(kclusters)
ys= [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors=[]
for lat, lon, poi, cluster, in zip(Queens_merged['Latitude'], Queens_merged['Longitude'], Queens_merged['Neighborhood'], Queens_merged["Cluster Labels"]):
    label = folium.Popup(str(poi)+ " Cluster "+ str(cluster), parse_html=True)
    folium.CircleMarker(
    [lat,lon],
    radius=5,
    popup=label,
    color = rainbow[cluster-1],
    fill= False).add_to(map_clusters)
    
map_clusters

## I.4. Examining the cluster 

In [281]:
#cluster 1
Queens_merged.loc[Queens_merged["Cluster Labels"] == 0, Queens_merged.columns[[1]+ list(range(5, Queens_merged.shape[1]))]]

,Neighborhood,1th Most Common venue,2th Most Common venue,3th Most Common venue,4th Most Common venue,5th Most Common venue,6th Most Common venue,7th Most Common venue,8th Most Common venue,9th Most Common venue,10th Most Common venue
43,Breezy Point,Beach,Supermarket,Monument / Landmark,Bus Stop,Trail,Farmers Market,Event Service,Event Space,Falafel Restaurant,Farm
50,Neponsit,Beach,Women's Store,Farmers Market,Empanada Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Food Court
78,Hammels,Beach,Building,Deli / Bodega,Bus Station,Food Truck,Diner,Bus Stop,Fried Chicken Joint,Dog Run,Fast Food Restaurant


In [282]:
#cluster 2
Queens_merged.loc[Queens_merged["Cluster Labels"] == 1, Queens_merged.columns[[1]+ list(range(5, Queens_merged.shape[1]))]]


,Neighborhood,1th Most Common venue,2th Most Common venue,3th Most Common venue,4th Most Common venue,5th Most Common venue,6th Most Common venue,7th Most Common venue,8th Most Common venue,9th Most Common venue,10th Most Common venue
79,Bayswater,Playground,Women's Store,Farm,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Event Space,Falafel Restaurant,Farmers Market


In [283]:
#cluster 3
Queens_merged.loc[Queens_merged["Cluster Labels"] == 2, Queens_merged.columns[[1]+ list(range(5, Queens_merged.shape[1]))]]

,Neighborhood,1th Most Common venue,2th Most Common venue,3th Most Common venue,4th Most Common venue,5th Most Common venue,6th Most Common venue,7th Most Common venue,8th Most Common venue,9th Most Common venue,10th Most Common venue
15,Glendale,Food & Drink Shop,Brewery,Pizza Place,Deli / Bodega,Arts & Crafts Store,Farmers Market,Empanada Restaurant,Event Service,Event Space,Falafel Restaurant
21,Whitestone,Bubble Tea Shop,Dance Studio,Deli / Bodega,Candy Store,Women's Store,Farmers Market,Event Service,Event Space,Falafel Restaurant,Farm
30,Briarwood,Deli / Bodega,Plaza,Coffee Shop,Gym,Bus Station,Arts & Crafts Store,Convenience Store,Indian Restaurant,Playground,Filipino Restaurant
42,Broad Channel,Sporting Goods Shop,Pizza Place,Deli / Bodega,Other Nightlife,Dive Bar,Women's Store,Falafel Restaurant,Electronics Store,Empanada Restaurant,Event Service
64,Brookville,Deli / Bodega,Women's Store,Farmers Market,Electronics Store,Empanada Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant
66,North Corona,Deli / Bodega,Gym / Fitness Center,Bakery,Pizza Place,Mobile Phone Shop,Caribbean Restaurant,Museum,Food Truck,Coffee Shop,Spanish Restaurant
69,Utopia,Deli / Bodega,Indie Movie Theater,Automotive Shop,Spa,Basketball Court,Pizza Place,South American Restaurant,Donut Shop,Ice Cream Shop,History Museum


In [284]:
#cluster 4
Queens_merged.loc[Queens_merged["Cluster Labels"] == 3, Queens_merged.columns[[1]+ list(range(5, Queens_merged.shape[1]))]]

,Neighborhood,1th Most Common venue,2th Most Common venue,3th Most Common venue,4th Most Common venue,5th Most Common venue,6th Most Common venue,7th Most Common venue,8th Most Common venue,9th Most Common venue,10th Most Common venue
0,Astoria,Bar,Middle Eastern Restaurant,Café,Hookah Bar,Greek Restaurant,Mediterranean Restaurant,Seafood Restaurant,Indian Restaurant,Bakery,Food Truck
1,Woodside,Grocery Store,Filipino Restaurant,Bakery,Latin American Restaurant,Thai Restaurant,Donut Shop,American Restaurant,Pub,Bar,Spa
2,Jackson Heights,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mobile Phone Shop,Thai Restaurant,Grocery Store,Pizza Place,Spanish Restaurant,Supermarket
3,Elmhurst,Thai Restaurant,Mexican Restaurant,Vietnamese Restaurant,Chinese Restaurant,Snack Place,Bubble Tea Shop,South American Restaurant,Seafood Restaurant,Malay Restaurant,Salon / Barbershop
4,Howard Beach,Italian Restaurant,Pharmacy,Fast Food Restaurant,Sandwich Place,Jewelry Store,Clothing Store,Japanese Restaurant,Sushi Restaurant,Donut Shop,Supermarket
...,...,...,...,...,...,...,...,...,...,...,...
74,Blissville,Deli / Bodega,Donut Shop,Mattress Store,Intersection,Bus Station,Sporting Goods Shop,Movie Theater,Restaurant,Clothing Store,Bakery
75,Roxbury,Irish Pub,Fast Food Restaurant,Beach,Trail,Baseball Field,Deli / Bodega,Fish Market,Fish & Chips Shop,Filipino Restaurant,Electronics Store
76,Middle Village,Pizza Place,Park,Bank,Bakery,Japanese Restaurant,Italian Restaurant,Sports Bar,Sushi Restaurant,Dessert Shop,Sandwich Place
77,Malba,Tennis Court,Rest Area,Rock Club,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Women's Store


## II. Staten Island

In [24]:
address = 'Staten Island, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location   = geolocator.geocode(address)
latitude   = location.latitude
longitude  = location.longitude

In [25]:
#Map the neighborhood
map_statenIsland = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, label in zip(statenIsland["Latitude"], statenIsland["Longitude"], statenIsland["Neighborhood"]):
    label = folium.Popup(label, parse_html= True)
    folium.CircleMarker(
    [lat, lng],
    radius = 5,
    popup = label,
    color="blue",
    fill = False,
    parse_html=False).add_to(map_statenIsland)
    
map_statenIsland

In [26]:
# get the neighborhood's name with their latitude and longitude 
neighborhood_name = statenIsland.loc[0,"Neighborhood"]
neighborhood_latitude = statenIsland.loc[0, "Latitude"]
neighborhood_longitude= statenIsland.loc[0, "Longitude"]

let get the top 100 venues that are in Marble Hill within a radius of 500 m

In [27]:
# get the URL
LIMIT = 100
radius= 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
#get the result
results = requests.get(url).json()

In [28]:
# put the json data into a pandas dataframe
venues = results["response"]["groups"][0]['items']
nearby_venues = json_normalize(venues)

filtered_columns = ["venue.name", "venue.categories","venue.location.lat","venue.location.lng"]
nearby_venues    = nearby_venues.loc[:, filtered_columns]

nearby_venues["venue.categories"] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns =[col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Beso,Tapas Restaurant,40.643306,-74.076508
1,A&S Pizzeria,Pizza Place,40.643940,-74.077626
2,Richmond County Bank Ballpark,Baseball Stadium,40.645056,-74.076864
3,Staten Island September 11 Memorial,Monument / Landmark,40.646767,-74.076510
4,Shake Shack,Burger Joint,40.643660,-74.075891


## I.1. Explore the neighborhood

In [29]:
# for each neighborhood
statenIsland_venues = getNearbyVenues(statenIsland["Neighborhood"], statenIsland["Latitude"], statenIsland["Longitude"])

St. George
New Brighton
Stapleton
Rosebank
West Brighton
Grymes Hill
Todt Hill
South Beach
Port Richmond
Mariner's Harbor
Port Ivory
Castleton Corners
New Springville
Travis
New Dorp
Oakwood
Great Kills
Eltingville
Annadale
Woodrow
Tottenville
Tompkinsville
Silver Lake
Sunnyside
Park Hill
Westerleigh
Graniteville
Arlington
Arrochar
Grasmere
Old Town
Dongan Hills
Midland Beach
Grant City
New Dorp Beach
Bay Terrace
Huguenot
Pleasant Plains
Butler Manor
Charleston
Rossville
Arden Heights
Greenridge
Heartland Village
Chelsea
Bloomfield
Bulls Head
Richmond Town
Shore Acres
Clifton
Concord
Emerson Hill
Randall Manor
Howland Hook
Elm Park
Manor Heights
Willowbrook
Sandy Ground
Egbertville
Prince's Bay
Lighthouse Hill
Richmond Valley
Fox Hills


In [30]:
print(statenIsland_venues.shape)
statenIsland_venues.head()

(836, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,St. George,40.644982,-74.079353,Beso,40.643306,-74.076508,Tapas Restaurant
1,St. George,40.644982,-74.079353,A&S Pizzeria,40.643940,-74.077626,Pizza Place
2,St. George,40.644982,-74.079353,Richmond County Bank Ballpark,40.645056,-74.076864,Baseball Stadium
3,St. George,40.644982,-74.079353,Staten Island September 11 Memorial,40.646767,-74.076510,Monument / Landmark
4,St. George,40.644982,-74.079353,Shake Shack,40.643660,-74.075891,Burger Joint


## I.2. Analyse each neighborhood

In [31]:
#one hot encoding
statenIsland_onehot = pd.get_dummies(statenIsland_venues[["Venue Category"]], prefix="", prefix_sep="")

#add neighborhood column back to to dataframe
statenIsland_onehot["Neighborhood"]= statenIsland_venues["Neighborhood"]
fixed_columns = [statenIsland_onehot.columns[-1]]+ list(statenIsland_onehot.columns[:-1])
statenIsland_onehot = statenIsland_onehot[fixed_columns]

print(statenIsland_onehot.shape)
statenIsland_onehot.head()

(836, 182)


,Neighborhood,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,St. George,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,St. George,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,St. George,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,St. George,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,St. George,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# grouping rows by neighborhood
statenIsland_grouped = statenIsland_onehot.groupby("Neighborhood").mean().reset_index()
print(statenIsland_grouped.shape)
statenIsland_grouped.head()

(61, 182)


,Neighborhood,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Annadale,0.142857,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,...,0.0,0.0,0.071429,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Arden Heights,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Arlington,0.166667,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Arrochar,0.000000,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.05,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bay Terrace,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
num_top_venues = 5

for i in statenIsland_grouped["Neighborhood"]:
    print("---" + i + "---")
    temp = statenIsland_grouped[statenIsland_grouped["Neighborhood"]==i].T.reset_index()
    temp.columns = ["venue", "freq"]
    temp = temp.iloc[1:]
    temp["freq"]= temp["freq"].astype(float)
    temp= temp.round({"freq":2})
    print (temp.sort_values("freq", ascending=False).reset_index(drop=True).head(num_top_venues))
    print("\n")

---Annadale---
                 venue  freq
0  American Restaurant  0.14
1               Bakery  0.14
2          Pizza Place  0.14
3         Liquor Store  0.07
4                 Park  0.07


---Arden Heights---
         venue  freq
0     Bus Stop   0.2
1       Lawyer   0.2
2  Coffee Shop   0.2
3  Pizza Place   0.2
4     Pharmacy   0.2


---Arlington---
                 venue  freq
0             Bus Stop  0.33
1  American Restaurant  0.17
2        Boat or Ferry  0.17
3         Intersection  0.17
4        Deli / Bodega  0.17


---Arrochar---
                venue  freq
0  Italian Restaurant  0.10
1            Bus Stop  0.10
2       Deli / Bodega  0.10
3         Pizza Place  0.10
4        Liquor Store  0.05


---Bay Terrace---
                venue  freq
0         Supermarket   0.2
1        Home Service   0.1
2               Plaza   0.1
3          Donut Shop   0.1
4  Salon / Barbershop   0.1


---Bloomfield---
               venue  freq
0  Recreation Center  0.25
1         Theme Park  0.2

In [34]:
num_top_venues = 10
indicators = ["st", "nd", "rd"]

columns = ["Neighborhood"]
for i in np.arange(num_top_venues):
    try:
        columns.append("{}{} Most Common Venue".format(i+1, incdicators[i]))
    except:
        columns.append("{}th Most Common venue".format(i+1))

# create dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted["Neighborhood"] = statenIsland_grouped["Neighborhood"]

for i in np.arange(statenIsland_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[i, 1:] = return_most_common_venues(statenIsland_grouped.iloc[i, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1th Most Common venue,2th Most Common venue,3th Most Common venue,4th Most Common venue,5th Most Common venue,6th Most Common venue,7th Most Common venue,8th Most Common venue,9th Most Common venue,10th Most Common venue
0,Annadale,American Restaurant,Bakery,Pizza Place,Cosmetics Shop,Restaurant,Pharmacy,Diner,Park,Liquor Store,Dance Studio
1,Arden Heights,Lawyer,Bus Stop,Pizza Place,Pharmacy,Coffee Shop,Eastern European Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market
2,Arlington,Bus Stop,American Restaurant,Boat or Ferry,Deli / Bodega,Intersection,Dim Sum Restaurant,Diner,Food,Flower Shop,Dessert Shop
3,Arrochar,Deli / Bodega,Pizza Place,Bus Stop,Italian Restaurant,Liquor Store,Polish Restaurant,Outdoors & Recreation,Sandwich Place,Food Truck,Supermarket
4,Bay Terrace,Supermarket,Home Service,Italian Restaurant,Salon / Barbershop,Shipping Store,Donut Shop,Sushi Restaurant,Plaza,Insurance Office,Harbor / Marina


## I.3. Cluster neighborhoods

In [35]:
kclusters = 3

statenIsland_grouped_clustering = statenIsland_grouped.drop("Neighborhood", 1)

# rum Kmean clustering
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(statenIsland_grouped_clustering)

#add labels
neighborhoods_venues_sorted.insert(0, "Cluster Labels", kmeans.labels_)
statenIsland_merged = statenIsland

statenIsland_merged = statenIsland_merged.join(neighborhoods_venues_sorted.set_index("Neighborhood"), on="Neighborhood")
statenIsland_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1th Most Common venue,2th Most Common venue,3th Most Common venue,4th Most Common venue,5th Most Common venue,6th Most Common venue,7th Most Common venue,8th Most Common venue,9th Most Common venue,10th Most Common venue
0,Staten Island,St. George,40.644982,-74.079353,1.0,Clothing Store,Sporting Goods Shop,Bar,Italian Restaurant,Scenic Lookout,Fast Food Restaurant,Farmers Market,Burger Joint,Falafel Restaurant,Bus Stop
1,Staten Island,New Brighton,40.640615,-74.087017,0.0,Bus Stop,Deli / Bodega,Park,Bowling Alley,Playground,Discount Store,Food,Event Space,Flower Shop,Fish & Chips Shop
2,Staten Island,Stapleton,40.626928,-74.077902,1.0,Pizza Place,Bank,Discount Store,Coffee Shop,Sandwich Place,Mexican Restaurant,Spanish Restaurant,Bus Stop,Skate Park,Café
3,Staten Island,Rosebank,40.615305,-74.069805,1.0,Grocery Store,Pizza Place,Italian Restaurant,Museum,Breakfast Spot,Burger Joint,Mexican Restaurant,Cajun / Creole Restaurant,Filipino Restaurant,American Restaurant
4,Staten Island,West Brighton,40.631879,-74.107182,1.0,Coffee Shop,Pharmacy,Bank,Breakfast Spot,Bar,Italian Restaurant,Music Store,Salon / Barbershop,Sandwich Place,Fast Food Restaurant


In [36]:
# create a map
map_clusters = folium.Map(Location=[latitude, longitude], zoom_start= 10)

x= np.arange(kclusters)
ys= [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors=[]
for lat, lon, poi, cluster, in zip(statenIsland_merged['Latitude'], statenIsland_merged['Longitude'], statenIsland_merged['Neighborhood'], statenIsland_merged["Cluster Labels"]):
    label = folium.Popup(str(poi)+ " Cluster "+ str(cluster), parse_html=True)
    folium.CircleMarker(
    [lat,lon],
    radius=5,
    popup=label,
    color = rainbow[cluster-1],
    fill= False).add_to(map_clusters)
    
#map_clusters

TypeError: list indices must be integers or slices, not float

## I.4. Examining the cluster 

In [357]:
#cluster 1
statenIsland_merged.loc[statenIsland_merged["Cluster Labels"] == 0, statenIsland_merged.columns[[1]+ list(range(5, statenIsland_merged.shape[1]))]]

,Neighborhood,1th Most Common venue,2th Most Common venue,3th Most Common venue,4th Most Common venue,5th Most Common venue,6th Most Common venue,7th Most Common venue,8th Most Common venue,9th Most Common venue,10th Most Common venue
1,New Brighton,Bus Stop,Park,Deli / Bodega,Discount Store,Bowling Alley,Daycare,Food,Playground,Event Space,Fish & Chips Shop
7,South Beach,Pier,Deli / Bodega,Bus Stop,Beach,Athletics & Sports,Yoga Studio,Electronics Store,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant
15,Oakwood,Lawyer,Nightlife Spot,Bar,Yoga Studio,Eastern European Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
24,Park Hill,Bus Stop,Hotel,Restaurant,Coffee Shop,Athletics & Sports,Gym / Fitness Center,Dry Cleaner,Filipino Restaurant,Fast Food Restaurant,Farmers Market
27,Arlington,American Restaurant,Deli / Bodega,Intersection,Bus Stop,Grocery Store,Yoga Studio,Eastern European Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant
41,Arden Heights,Deli / Bodega,Pizza Place,Coffee Shop,Bus Stop,Pharmacy,Business Service,Lawyer,Eastern European Restaurant,Filipino Restaurant,Fast Food Restaurant
44,Chelsea,Steakhouse,Spanish Restaurant,Bus Stop,Sandwich Place,Park,Donut Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
45,Bloomfield,Recreation Center,Burger Joint,Bus Stop,Theme Park,Flower Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
56,Willowbrook,Bus Stop,Chinese Restaurant,Intersection,Pizza Place,Spa,Bagel Shop,Yoga Studio,Eastern European Restaurant,Filipino Restaurant,Fast Food Restaurant
57,Sandy Ground,Bus Stop,Racetrack,Food Truck,Historic Site,Art Gallery,Intersection,Playground,Clothing Store,Fish & Chips Shop,Bar


In [358]:
#cluster 2
statenIsland_merged.loc[statenIsland_merged["Cluster Labels"] == 1, statenIsland_merged.columns[[1]+ list(range(5, statenIsland_merged.shape[1]))]]

,Neighborhood,1th Most Common venue,2th Most Common venue,3th Most Common venue,4th Most Common venue,5th Most Common venue,6th Most Common venue,7th Most Common venue,8th Most Common venue,9th Most Common venue,10th Most Common venue
5,Grymes Hill,Dog Run,Yoga Studio,Dry Cleaner,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store


In [359]:
#cluster 3
statenIsland_merged.loc[statenIsland_merged["Cluster Labels"] == 2, statenIsland_merged.columns[[1]+ list(range(5, statenIsland_merged.shape[1]))]]

,Neighborhood,1th Most Common venue,2th Most Common venue,3th Most Common venue,4th Most Common venue,5th Most Common venue,6th Most Common venue,7th Most Common venue,8th Most Common venue,9th Most Common venue,10th Most Common venue
0,St. George,Clothing Store,Italian Restaurant,Sporting Goods Shop,Bar,Coffee Shop,Burger Joint,Baseball Stadium,Steakhouse,Farmers Market,Scenic Lookout
2,Stapleton,Pizza Place,Sandwich Place,Discount Store,Bank,Pool Hall,Fast Food Restaurant,Spanish Restaurant,Mexican Restaurant,Skate Park,Café
3,Rosebank,Pizza Place,Italian Restaurant,Grocery Store,Cosmetics Shop,Park,Discount Store,Donut Shop,Restaurant,Beach,Eastern European Restaurant
4,West Brighton,Coffee Shop,Breakfast Spot,Italian Restaurant,Pharmacy,American Restaurant,Music Store,Bar,Bank,Food & Drink Shop,Burger Joint
6,Todt Hill,Trail,Park,Yoga Studio,Donut Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store
8,Port Richmond,Pizza Place,Donut Shop,Rental Car Location,Bus Stop,Bar,Dry Cleaner,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
9,Mariner's Harbor,Italian Restaurant,Deli / Bodega,Supermarket,Home Service,Dry Cleaner,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
10,Port Ivory,Recording Studio,Dry Cleaner,Flower Shop,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store
11,Castleton Corners,Pizza Place,Bank,Ice Cream Shop,Grocery Store,Skating Rink,Sandwich Place,Tattoo Parlor,Go Kart Track,Bagel Shop,Bar
12,New Springville,Chinese Restaurant,Pizza Place,Ice Cream Shop,Martial Arts School,Restaurant,Pharmacy,Optical Shop,Mobile Phone Shop,Mexican Restaurant,Liquor Store


## III. Manhattan

In [37]:
address = 'Manhattan, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location   = geolocator.geocode(address)
latitude   = location.latitude
longitude  = location.longitude

In [38]:
#Map the neighborhood
map_Manhattan = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, label in zip(manhattan_data["Latitude"], manhattan_data["Longitude"], manhattan_data["Neighborhood"]):
    label = folium.Popup(label, parse_html= True)
    folium.CircleMarker(
    [lat, lng],
    radius = 5,
    popup = label,
    color="blue",
    fill = False,
    parse_html=False).add_to(map_Manhattan)
    
map_Manhattan

In [39]:
# get the neighborhood's name with their latitude and longitude 
neighborhood_name = manhattan_data.loc[0,"Neighborhood"]
neighborhood_latitude = manhattan_data.loc[0, "Latitude"]
neighborhood_longitude= manhattan_data.loc[0, "Longitude"]

let get the top 100 venues that are in Marble Hill within a radius of 500 m

In [40]:
# get the URL
LIMIT = 100
radius= 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
#get the result
results = requests.get(url).json()

In [41]:
# put the json data into a pandas dataframe
venues = results["response"]["groups"][0]['items']
nearby_venues = json_normalize(venues)

filtered_columns = ["venue.name", "venue.categories","venue.location.lat","venue.location.lng"]
nearby_venues    = nearby_venues.loc[:, filtered_columns]

nearby_venues["venue.categories"] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns =[col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Starbucks,Coffee Shop,40.877531,-73.905582
4,Dunkin',Donut Shop,40.877136,-73.906666


## I.1. Explore the neighborhood

In [42]:
# for each neighborhood
manhattan_venues = getNearbyVenues(manhattan_data["Neighborhood"], manhattan_data["Latitude"], manhattan_data["Longitude"])

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [43]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3192, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


## I.2. Analyse each neighborhood

In [45]:
#one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[["Venue Category"]], prefix="", prefix_sep="")

#add neighborhood column back to to dataframe
manhattan_onehot["Neighborhood"]= manhattan_venues["Neighborhood"]
fixed_columns = [manhattan_onehot.columns[-1]]+ list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

print(manhattan_onehot.shape)
manhattan_onehot.head()

(3192, 330)


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
# grouping rows by neighborhood
manhattan_grouped = manhattan_onehot.groupby("Neighborhood").mean().reset_index()
print(manhattan_grouped.shape)
manhattan_grouped.head()

(40, 330)


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Battery Park City,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.013889,0.0,0.000000,0.000000
1,Carnegie Hill,0.0,0.0,0.0,0.000000,0.011236,0.0,0.00,0.011236,0.000000,...,0.0,0.011236,0.0,0.0,0.0,0.011236,0.033708,0.0,0.011236,0.033708
2,Central Harlem,0.0,0.0,0.0,0.068182,0.045455,0.0,0.00,0.000000,0.045455,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
3,Chelsea,0.0,0.0,0.0,0.000000,0.040000,0.0,0.01,0.000000,0.060000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.030000,0.0,0.010000,0.010000
4,Chinatown,0.0,0.0,0.0,0.000000,0.030000,0.0,0.00,0.000000,0.000000,...,0.0,0.020000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.010000


In [47]:
num_top_venues = 5

for i in manhattan_grouped["Neighborhood"]:
    print("---" + i + "---")
    temp = manhattan_grouped[manhattan_grouped["Neighborhood"]==i].T.reset_index()
    temp.columns = ["venue", "freq"]
    temp = temp.iloc[1:]
    temp["freq"]= temp["freq"].astype(float)
    temp= temp.round({"freq":2})
    print (temp.sort_values("freq", ascending=False).reset_index(drop=True).head(num_top_venues))
    print("\n")

---Battery Park City---
            venue  freq
0     Coffee Shop  0.08
1            Park  0.08
2           Hotel  0.07
3  Clothing Store  0.06
4   Memorial Site  0.04


---Carnegie Hill---
         venue  freq
0  Coffee Shop  0.09
1         Café  0.04
2          Bar  0.04
3  Yoga Studio  0.03
4  Pizza Place  0.03


---Central Harlem---
                venue  freq
0  African Restaurant  0.07
1      Cosmetics Shop  0.07
2  Seafood Restaurant  0.05
3         Art Gallery  0.05
4   French Restaurant  0.05


---Chelsea---
                 venue  freq
0          Coffee Shop  0.07
1          Art Gallery  0.06
2               Bakery  0.05
3  American Restaurant  0.04
4                 Café  0.03


---Chinatown---
                 venue  freq
0               Bakery  0.08
1   Chinese Restaurant  0.07
2         Cocktail Bar  0.05
3                  Spa  0.03
4  American Restaurant  0.03


---Civic Center---
                  venue  freq
0           Coffee Shop  0.08
1                   Spa  0.05


In [48]:
num_top_venues = 10
indicators = ["st", "nd", "rd"]

columns = ["Neighborhood"]
for i in np.arange(num_top_venues):
    try:
        columns.append("{}{} Most Common Venue".format(i+1, incdicators[i]))
    except:
        columns.append("{}th Most Common venue".format(i+1))

# create dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted["Neighborhood"] = manhattan_grouped["Neighborhood"]

for i in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[i, 1:] = return_most_common_venues(manhattan_grouped.iloc[i, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1th Most Common venue,2th Most Common venue,3th Most Common venue,4th Most Common venue,5th Most Common venue,6th Most Common venue,7th Most Common venue,8th Most Common venue,9th Most Common venue,10th Most Common venue
0,Battery Park City,Park,Coffee Shop,Hotel,Clothing Store,Gym,Memorial Site,Plaza,Playground,Shopping Mall,Burger Joint
1,Carnegie Hill,Coffee Shop,Café,Bar,Yoga Studio,Wine Shop,Bookstore,Gym / Fitness Center,Gym,Pizza Place,Cosmetics Shop
2,Central Harlem,Cosmetics Shop,African Restaurant,Seafood Restaurant,Bar,American Restaurant,French Restaurant,Chinese Restaurant,Art Gallery,Library,Ethiopian Restaurant
3,Chelsea,Coffee Shop,Art Gallery,Bakery,American Restaurant,Ice Cream Shop,Seafood Restaurant,Café,French Restaurant,Wine Shop,Cycle Studio
4,Chinatown,Bakery,Chinese Restaurant,Cocktail Bar,Optical Shop,American Restaurant,Spa,Salon / Barbershop,Bubble Tea Shop,Ice Cream Shop,Hotpot Restaurant


## I.3. Cluster neighborhoods

In [49]:
kclusters = 7

manhattan_grouped_clustering = manhattan_grouped.drop("Neighborhood", 1)

# rum Kmean clustering
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(manhattan_grouped_clustering)

#add labels
neighborhoods_venues_sorted.insert(0, "Cluster Labels", kmeans.labels_)
manhattan_merged = manhattan_data

manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index("Neighborhood"), on="Neighborhood")
manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1th Most Common venue,2th Most Common venue,3th Most Common venue,4th Most Common venue,5th Most Common venue,6th Most Common venue,7th Most Common venue,8th Most Common venue,9th Most Common venue,10th Most Common venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Gym,Discount Store,Coffee Shop,Sandwich Place,Seafood Restaurant,Tennis Stadium,Big Box Store,Supplement Shop,Steakhouse,Shopping Mall
1,Manhattan,Chinatown,40.715618,-73.994279,6,Bakery,Chinese Restaurant,Cocktail Bar,Optical Shop,American Restaurant,Spa,Salon / Barbershop,Bubble Tea Shop,Ice Cream Shop,Hotpot Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,6,Café,Grocery Store,Bakery,Mobile Phone Shop,Gym,Chinese Restaurant,Sandwich Place,Tapas Restaurant,Coffee Shop,Latin American Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,6,Mexican Restaurant,Café,Restaurant,Lounge,Chinese Restaurant,Frozen Yogurt Shop,Spanish Restaurant,Park,Bakery,Pizza Place
4,Manhattan,Hamilton Heights,40.823604,-73.949688,6,Pizza Place,Deli / Bodega,Café,Coffee Shop,Mexican Restaurant,Park,Caribbean Restaurant,School,Chinese Restaurant,Bakery


In [50]:
# create a map
map_clusters = folium.Map(Location=[latitude, longitude], zoom_start= 10)

x= np.arange(kclusters)
ys= [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors=[]
for lat, lon, poi, cluster, in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged["Cluster Labels"]):
    label = folium.Popup(str(poi)+ " Cluster "+ str(cluster), parse_html=True)
    folium.CircleMarker(
    [lat,lon],
    radius=5,
    popup=label,
    color = rainbow[cluster-1],
    fill= False).add_to(map_clusters)
    
map_clusters

## I.4. Examining the cluster 


In [53]:
#cluster 1
manhattan_merged.loc[manhattan_merged["Cluster Labels"] == 0, manhattan_merged.columns[[1]+ list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1th Most Common venue,2th Most Common venue,3th Most Common venue,4th Most Common venue,5th Most Common venue,6th Most Common venue,7th Most Common venue,8th Most Common venue,9th Most Common venue,10th Most Common venue
6,Central Harlem,Cosmetics Shop,African Restaurant,Seafood Restaurant,Bar,American Restaurant,French Restaurant,Chinese Restaurant,Art Gallery,Library,Ethiopian Restaurant
13,Lincoln Square,Plaza,Café,Theater,Italian Restaurant,Performing Arts Venue,Concert Hall,French Restaurant,Indie Movie Theater,Gym / Fitness Center,Wine Shop


In [54]:
#cluster 2
manhattan_merged.loc[manhattan_merged["Cluster Labels"] == 1, manhattan_merged.columns[[1]+ list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1th Most Common venue,2th Most Common venue,3th Most Common venue,4th Most Common venue,5th Most Common venue,6th Most Common venue,7th Most Common venue,8th Most Common venue,9th Most Common venue,10th Most Common venue
8,Upper East Side,Exhibit,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Gym / Fitness Center,Yoga Studio,French Restaurant,Spa,Juice Bar
9,Yorkville,Italian Restaurant,Coffee Shop,Bar,Gym,Deli / Bodega,Sushi Restaurant,Wine Shop,Japanese Restaurant,Diner,Pharmacy
10,Lenox Hill,Italian Restaurant,Sushi Restaurant,Coffee Shop,Pizza Place,Café,Cocktail Bar,Burger Joint,Gym / Fitness Center,Gym,Mexican Restaurant
12,Upper West Side,Italian Restaurant,Wine Bar,Bar,Coffee Shop,Café,Bakery,Mexican Restaurant,Pub,Pizza Place,Bagel Shop
16,Murray Hill,Sandwich Place,Coffee Shop,Hotel,American Restaurant,Pizza Place,Japanese Restaurant,Burger Joint,Gym / Fitness Center,Bar,Furniture / Home Store
17,Chelsea,Coffee Shop,Art Gallery,Bakery,American Restaurant,Ice Cream Shop,Seafood Restaurant,Café,French Restaurant,Wine Shop,Cycle Studio
18,Greenwich Village,Italian Restaurant,Sushi Restaurant,Clothing Store,Café,American Restaurant,Gym,Bakery,Dessert Shop,Boutique,Chinese Restaurant
19,East Village,Bar,Pizza Place,Mexican Restaurant,Salon / Barbershop,Cocktail Bar,Coffee Shop,Vietnamese Restaurant,Korean Restaurant,Japanese Restaurant,Gourmet Shop
22,Little Italy,Café,Bakery,Italian Restaurant,Cocktail Bar,Chinese Restaurant,Clothing Store,Salon / Barbershop,Sandwich Place,Bubble Tea Shop,Mediterranean Restaurant
23,Soho,Clothing Store,Italian Restaurant,Mediterranean Restaurant,Boutique,Coffee Shop,Hotel,Sporting Goods Shop,Salon / Barbershop,Bakery,French Restaurant


In [55]:
#cluster 3
manhattan_merged.loc[manhattan_merged["Cluster Labels"] == 2, manhattan_merged.columns[[1]+ list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1th Most Common venue,2th Most Common venue,3th Most Common venue,4th Most Common venue,5th Most Common venue,6th Most Common venue,7th Most Common venue,8th Most Common venue,9th Most Common venue,10th Most Common venue
14,Clinton,Gym / Fitness Center,Italian Restaurant,Theater,American Restaurant,Gym,Coffee Shop,Spa,Hotel,Wine Shop,Sandwich Place
15,Midtown,Hotel,Clothing Store,Coffee Shop,Theater,Sporting Goods Shop,Bakery,Bookstore,Café,Steakhouse,French Restaurant
21,Tribeca,Park,Italian Restaurant,Wine Bar,American Restaurant,Bakery,Café,Spa,Coffee Shop,Hotel,Playground
28,Battery Park City,Park,Coffee Shop,Hotel,Clothing Store,Gym,Memorial Site,Plaza,Playground,Shopping Mall,Burger Joint
29,Financial District,Coffee Shop,Pizza Place,Mexican Restaurant,Park,Sandwich Place,Gym / Fitness Center,Gym,Café,Hotel,Cocktail Bar
30,Carnegie Hill,Coffee Shop,Café,Bar,Yoga Studio,Wine Shop,Bookstore,Gym / Fitness Center,Gym,Pizza Place,Cosmetics Shop
32,Civic Center,Coffee Shop,Cocktail Bar,Spa,Gym / Fitness Center,French Restaurant,Hotel,Park,Sushi Restaurant,Hotel Bar,Deli / Bodega
33,Midtown South,Korean Restaurant,Hotel,Japanese Restaurant,Dessert Shop,Burger Joint,American Restaurant,Bakery,Coffee Shop,Gym / Fitness Center,Flower Shop


In [56]:
#cluster 4
manhattan_merged.loc[manhattan_merged["Cluster Labels"] == 3, manhattan_merged.columns[[1]+ list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1th Most Common venue,2th Most Common venue,3th Most Common venue,4th Most Common venue,5th Most Common venue,6th Most Common venue,7th Most Common venue,8th Most Common venue,9th Most Common venue,10th Most Common venue
37,Stuyvesant Town,Park,Bar,Coffee Shop,Baseball Field,Pet Service,Gym / Fitness Center,Harbor / Marina,Cocktail Bar,Bistro,Heliport


In [203]:
#cluster 5
manhattan_merged.loc[manhattan_merged["Cluster Labels"] == 4, manhattan_merged.columns[[1]+ list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1th Most Common venue,2th Most Common venue,3th Most Common venue,4th Most Common venue,5th Most Common venue,6th Most Common venue,7th Most Common venue,8th Most Common venue,9th Most Common venue,10th Most Common venue
33,Midtown South,Korean Restaurant,Hotel,American Restaurant,Japanese Restaurant,Coffee Shop,Gym / Fitness Center,Burger Joint,Dessert Shop,Gym,Gastropub


In [58]:
#cluster 6
manhattan_merged.loc[manhattan_merged["Cluster Labels"] == 5, manhattan_merged.columns[[1]+ list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1th Most Common venue,2th Most Common venue,3th Most Common venue,4th Most Common venue,5th Most Common venue,6th Most Common venue,7th Most Common venue,8th Most Common venue,9th Most Common venue,10th Most Common venue
11,Roosevelt Island,Park,Food & Drink Shop,Residential Building (Apartment / Condo),Scenic Lookout,Supermarket,Sandwich Place,Liquor Store,Soccer Field,Baseball Field,Coffee Shop
26,Morningside Heights,Park,Bookstore,American Restaurant,Coffee Shop,Café,Burger Joint,Food Truck,Mediterranean Restaurant,Ethiopian Restaurant,New American Restaurant
35,Turtle Bay,Coffee Shop,Italian Restaurant,Deli / Bodega,Park,Sushi Restaurant,Japanese Restaurant,Ramen Restaurant,Seafood Restaurant,Pharmacy,Karaoke Bar
36,Tudor City,Park,Café,Mexican Restaurant,Coffee Shop,Deli / Bodega,Diner,Greek Restaurant,Sushi Restaurant,Spanish Restaurant,Thai Restaurant


In [59]:
#cluster 7
manhattan_merged.loc[manhattan_merged["Cluster Labels"] == 6, manhattan_merged.columns[[1]+ list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1th Most Common venue,2th Most Common venue,3th Most Common venue,4th Most Common venue,5th Most Common venue,6th Most Common venue,7th Most Common venue,8th Most Common venue,9th Most Common venue,10th Most Common venue
1,Chinatown,Bakery,Chinese Restaurant,Cocktail Bar,Optical Shop,American Restaurant,Spa,Salon / Barbershop,Bubble Tea Shop,Ice Cream Shop,Hotpot Restaurant
2,Washington Heights,Café,Grocery Store,Bakery,Mobile Phone Shop,Gym,Chinese Restaurant,Sandwich Place,Tapas Restaurant,Coffee Shop,Latin American Restaurant
3,Inwood,Mexican Restaurant,Café,Restaurant,Lounge,Chinese Restaurant,Frozen Yogurt Shop,Spanish Restaurant,Park,Bakery,Pizza Place
4,Hamilton Heights,Pizza Place,Deli / Bodega,Café,Coffee Shop,Mexican Restaurant,Park,Caribbean Restaurant,School,Chinese Restaurant,Bakery
5,Manhattanville,Seafood Restaurant,Coffee Shop,Mexican Restaurant,Italian Restaurant,Deli / Bodega,Falafel Restaurant,Grocery Store,Bike Trail,Sushi Restaurant,Lounge
7,East Harlem,Mexican Restaurant,Thai Restaurant,Bakery,Deli / Bodega,Spa,Sandwich Place,Latin American Restaurant,Beer Bar,Liquor Store,Taco Place
20,Lower East Side,Chinese Restaurant,Deli / Bodega,Pharmacy,Café,Bakery,Art Gallery,Coffee Shop,Japanese Restaurant,Sandwich Place,French Restaurant
